# Préparation des données

In [1]:
import os, sys

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark-0.90.jar,xgboost4j-0.90.jar pyspark-shell'

import findspark
findspark.init()

import pyspark
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


In [2]:
spark = SparkSession\
        .builder\
        .appName("PySpark XGBOOST")\
        .getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Ne peut attribuer l'adresse demandée: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:433)
	at sun.nio.ch.Net.bind(Net.java:425)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:223)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:128)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:558)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1283)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:501)
	at io.netty.channel.Ab

In [23]:
spark.sparkContext.addPyFile("/home/malick/Bureau/Stage_Projet_RTS/Spark/SparkXgboost/sparkxgb.zip")

In [16]:
spark.sparkContext.version

'2.4.3'

In [52]:
"""nous définissons un schéma des données que nous lisons à partir du csv. 
C'est généralement une meilleure pratique que de laisser spark pour déduire le schéma, 
car il consomme moins de ressources et nous avons un contrôle total sur les champs."""

#schema = StructType(
  #[StructField('_c0', IntegerType()),
   # StructField("Embarked", StringType())
  #])


"nous définissons un schéma des données que nous lisons à partir du csv. \nC'est généralement une meilleure pratique que de laisser spark pour déduire le schéma, \ncar il consomme moins de ressources et nous avons un contrôle total sur les champs."

In [24]:
file = "/home/malick/Bureau/Stage_Projet_RTS/confluence-JDD/airbnb/data"

df_listings = spark.read.format("csv").option("header", "true") \
                               .option("delimiter", ",") \
                               .option("inferSchema", "true") \
                               .load(file + "/listings.csv")

df_neighbourhoods = spark.read.format("csv").option("header", "true") \
                               .option("delimiter", ",") \
                               .option("inferSchema", "true") \
                               .load(file + "/neighbourhoods.csv")

df_reviews = spark.read.format("csv").option("header", "true") \
                               .option("delimiter", ",") \
                               .option("inferSchema", "true") \
                               .load(file + "/reviews.csv")

In [31]:
df_reviews.toPandas().head(5)

KeyboardInterrupt: 

[('AccountLength', 'int'),
 ('IntlPlan', 'string'),
 ('VMailPlan', 'string'),
 ('DayMins', 'double'),
 ('DayCalls', 'int'),
 ('DayCharge', 'string'),
 ('EveMins', 'string'),
 ('EveCalls', 'string'),
 ('EveCharge', 'double'),
 ('NightMins', 'double'),
 ('NightCalls', 'int'),
 ('NightCharge', 'string'),
 ('IntlMins', 'double'),
 ('IntlCalls', 'string'),
 ('IntlCharge', 'string'),
 ('CustServCalls', 'int'),
 ('Churn?', 'string')]

In [56]:
from pyspark.sql.types import IntegerType
stringCols = ['DayCharge','EveMins', 'EveCalls','NightCharge','IntlCalls','IntlCharge']
for i in stringCols:
    data = data.withColumn(i, data[i].cast(IntegerType()))
    
data.toPandas().head(10)

,AccountLength,IntlPlan,VMailPlan,DayMins,DayCalls,DayCharge,EveMins,EveCalls,EveCharge,NightMins,NightCalls,NightCharge,IntlMins,IntlCalls,IntlCharge,CustServCalls,Churn?
0,128,no,yes,265.1,110,NaN,197.0,99.0,16.78,244.7,91,11.0,10.0,3.0,2.0,1,False.
1,107,no,yes,161.6,123,NaN,195.0,103.0,16.62,254.4,103,11.0,13.7,3.0,3.0,1,False.
2,137,no,no,243.4,114,NaN,121.0,110.0,10.30,162.6,104,7.0,12.2,5.0,3.0,0,False.
3,84,yes,no,299.4,71,50.0,61.0,88.0,5.26,196.9,89,8.0,6.6,7.0,1.0,2,False.
4,75,yes,no,166.7,113,28.0,148.0,122.0,12.61,186.9,121,8.0,10.1,3.0,2.0,3,False.
5,118,yes,no,223.4,98,37.0,220.0,101.0,18.75,203.9,118,9.0,6.3,6.0,1.0,0,False.
6,121,no,yes,218.2,88,37.0,348.0,108.0,29.62,212.6,118,9.0,7.5,7.0,2.0,3,False.
7,147,yes,no,157.0,79,26.0,103.0,94.0,8.76,211.8,96,9.0,7.1,NaN,1.0,0,False.
8,117,no,no,184.5,97,31.0,351.0,80.0,29.89,215.8,90,9.0,8.7,4.0,NaN,1,False.
9,141,yes,yes,258.6,84,43.0,222.0,111.0,18.87,326.4,97,14.0,11.2,5.0,3.0,0,False.


In [57]:
# Count total NA's in each column
print(data.toPandas().isnull().sum())

AccountLength      0
IntlPlan           0
VMailPlan          0
DayMins            0
DayCalls           0
DayCharge        112
EveMins           88
EveCalls         520
EveCharge          0
NightMins          0
NightCalls         0
NightCharge      310
IntlMins           0
IntlCalls        662
IntlCharge       352
CustServCalls      0
Churn?             0
dtype: int64


Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/home/malick/Bureau/Stage_Projet_RTS/Calcul_distribues/spark-2.4.3-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'XGBoostClassifier' object has no attribute '_java_obj'


In [58]:
#On remplace les valeurs manquates par la moyenne des colonnes
from pyspark.sql.functions import mean
ColNaN = ["DayCharge", "EveMins", "EveCalls", "NightCharge", "IntlCalls", "IntlCharge"]
for col in ColNaN:
    moyenne = data.select(mean(col)).collect()[0][0]
    data = data.fillna( { col:moyenne} )
    
data.toPandas().head(10)

,AccountLength,IntlPlan,VMailPlan,DayMins,DayCalls,DayCharge,EveMins,EveCalls,EveCharge,NightMins,NightCalls,NightCharge,IntlMins,IntlCalls,IntlCharge,CustServCalls,Churn?
0,128,no,yes,265.1,110,30,197,99,16.78,244.7,91,11,10.0,3,2,1,False.
1,107,no,yes,161.6,123,30,195,103,16.62,254.4,103,11,13.7,3,3,1,False.
2,137,no,no,243.4,114,30,121,110,10.30,162.6,104,7,12.2,5,3,0,False.
3,84,yes,no,299.4,71,50,61,88,5.26,196.9,89,8,6.6,7,1,2,False.
4,75,yes,no,166.7,113,28,148,122,12.61,186.9,121,8,10.1,3,2,3,False.
5,118,yes,no,223.4,98,37,220,101,18.75,203.9,118,9,6.3,6,1,0,False.
6,121,no,yes,218.2,88,37,348,108,29.62,212.6,118,9,7.5,7,2,3,False.
7,147,yes,no,157.0,79,26,103,94,8.76,211.8,96,9,7.1,4,1,0,False.
8,117,no,no,184.5,97,31,351,80,29.89,215.8,90,9,8.7,4,2,1,False.
9,141,yes,yes,258.6,84,43,222,111,18.87,326.4,97,14,11.2,5,3,0,False.


In [59]:
print(data.toPandas().isnull().sum())

AccountLength    0
IntlPlan         0
VMailPlan        0
DayMins          0
DayCalls         0
DayCharge        0
EveMins          0
EveCalls         0
EveCharge        0
NightMins        0
NightCalls       0
NightCharge      0
IntlMins         0
IntlCalls        0
IntlCharge       0
CustServCalls    0
Churn?           0
dtype: int64


In [60]:
# on importe une classe qui transforme les colonnes qualitatives en colonnes entiers
# comme (equivalent de LabelEncoder de Scikit-Learn)
from pyspark.ml.feature import StringIndexer

ColString = ["IntlPlan","VMailPlan","Churn?"]

# méthode 1:
#indexer = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) for column in list(set(ColString)) ]

# méthode 2:
""" Avec l'option setHandleInvalid("keep"), l'indexeur ajoute de nouveaux index lorsqu'il 
voit de nouvelles étiquettes. Notez qu'avec les versions précédentes, vous avez également 
l'option "ignorer", ce qui permet à l'indexeur d'ignorer (supprimer) les lignes avec de 
nouvelles étiquettes."""

IntlPlanIndex = StringIndexer().setInputCol("IntlPlan").setOutputCol("IntlPlanIndex").setHandleInvalid("keep")
VMailPlanIndex = StringIndexer().setInputCol("VMailPlan").setOutputCol("VMailPlanIndex").setHandleInvalid("keep")
ChurnIndex = StringIndexer().setInputCol("Churn?").setOutputCol("Churn?Index").setHandleInvalid("keep")


In [61]:
#on construit ensuite un vecteur rassemblant toutes les colonnes explicatives
from pyspark.ml.feature import VectorAssembler

#on rassemble la liste des colonnes numériques que l'on va utiliser
numericsCols = [col for col in list(data.columns) if col not in ColString]
numericsCols += [col+"Index" for col in ColString][:-1]

# on crée un objet qui rassemble toutes ces colonnes dans une colonne nommée var_expl
assembler = VectorAssembler(inputCols=numericsCols, outputCol="var_expl")

In [86]:
from sparkxgb import XGBoostClassifier

# Define and train model
xgboost = XGBoostClassifier(
    
    # General Params
#    numWorkers=1, nthread=1, checkpointInterval=-1, checkpointPath="",
#   useExternalMemory=False, silent=0, missing=float("nan"),
    
    # Column Params
    labelCol="Churn?Index", featuresCol="var_expl", predictionCol="prediction", 
#    weightCol="weight", baseMarginCol="baseMargin", 
    
    # Booster Params
    baseScore=0.5, objective="binary:logistic", evalMetric="error", 
    numClass=2, numRound=2, seed=None,
    
    # Tree Booster Params
#   eta=0.3, maxDepth=8, minChildWeight=1.0, maxDeltaStep=0.0, subsample=1.0,
#   colsampleBytree=1.0, colsampleBylevel=1.0, reg_lambda=0.0, alpha=0.0, treeMethod="auto",
#    sketchEps=0.03, scalePosWeight=1.0, growPolicy='depthwise', maxBin=256,
)


In [87]:
from pyspark.ml import Pipeline

# on construit le pipeline
pipeline = Pipeline().setStages([IntlPlanIndex, VMailPlanIndex, ChurnIndex, assembler, xgboost])

In [88]:
# on divise le DataFrame initial (data) en deux DataFrame
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed=24)

In [89]:
# on ajuste le modéle
xgboost_model = pipeline.fit(trainingData)

Py4JJavaError: An error occurred while calling o2269.fit.
: java.lang.Exception: The number of classes in dataset doesn't match 'num_class' in xgboost params.
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.train(XGBoostClassifier.scala:172)
	at ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier.train(XGBoostClassifier.scala:44)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:118)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.G

In [82]:
# Transform test set
#xgboost_model.transform(testData).show()

In [83]:
# prévision sur les données de validation
predictions = xgboost_model.transform(testData)

In [84]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# cette classe calcule l'AUC de notre modéle
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", \
                                          labelCol="Churn?Index")

# on appliquenles données prédites à notre objet d'évaluation 
#evaluator.evaluate(predictions)

#l'AUC est affichée:

In [85]:
# on calcule l'accuracy manuellement
accuracy = predictions.filter(predictions["Churn?Index"] == predictions.prediction).count() \
           / float(testData.count())

accuracy
# on obtient un taux de:

0.8624873609706775

In [ ]:
# Write model/classifier
xgboost.write().overwrite().save("Models/xgboost_class_test")
xgboost_model.write().overwrite().save("Models/xgboost_class_test.model")

In [ ]:
# Load model

In [ ]:
spark.stop()